# 📘 Análisis Exploratorio Visual de Accidentes Viales en CDMX (2024)

Este notebook tiene como objetivo realizar un **análisis exploratorio visual (EDA)** del dataset limpio de accidentes de tránsito en la Ciudad de México. A través de gráficos interactivos generados con **Plotly**, se exploran patrones temporales, geográficos y categóricos de los siniestros viales.

Se analizan variables como:
- Fecha y hora del evento
- Alcaldía y colonia donde ocurrió
- Tipo de evento reportado
- Nivel de prioridad en la atención

El propósito de este análisis es **identificar tendencias y comportamientos clave** que servirán como base para el diseño del dashboard interactivo en Streamlit.


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

df = pd.read_csv('../data/accidentes_cdmx_limpio.csv', parse_dates=['fecha_evento'])

# Extracción de variables temporales

# Extrae el número de mes (1 a 12) y lo guarda en una nueva columna llamada 'mes'
df['mes'] = df['fecha_evento'].dt.month

# Obtiene el nombre del día de la semana y lo guarda en 'día_semana'
df['día_semana'] = df['fecha_evento'].dt.day_name()

# Extrae la hora (en formato 24h, de 0 a 23) y la guarda en la columna 'hora'
df['hora'] = df['fecha_evento'].dt.hour

In [2]:
# Gráfico de accidentes por mes. 

# Diccionario para traducir número de mes a nombre en español
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

# Cuenta accidentes por mes y ordena de enero a diciembre
conteo_meses = df['mes'].value_counts().sort_index()

# Convierte el número de mes en su nombre en español
conteo_meses.index = conteo_meses.index.map(meses_es)

# Crea gráfico de barras con nombres de meses como eje X
fig = px.bar(
    x=conteo_meses.index,
    y=conteo_meses.values,
    title="Accidentes por Mes",
    labels={'x': 'Mes', 'y': 'Número de accidentes'}
)

# Ajusta etiquetas de los ejes X y Y
fig.update_layout(xaxis_title='Mes', yaxis_title='Número de accidentes')

# Muestra el gráfico
fig.show()




- Los accidentes están relativamente distribuidos a lo largo del año. Sin embargo, los meses de **julio** y **septiembre** presentan una leve disminución.

In [3]:
# Gráfico de accidentes por día de la semana. 

# Orden correcto de los días en inglés (para mantener el orden cronológico)
orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Diccionario para traducir los días de inglés a español
dias_es = {
    'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles',
    'Thursday': 'Jueves', 'Friday': 'Viernes', 'Saturday': 'Sábado', 'Sunday': 'Domingo'
}

# Cuenta los accidentes por día y reordena según la semana
conteo_dias = df['día_semana'].value_counts().reindex(orden)

# Traduce los nombres de los días al español
conteo_dias.index = conteo_dias.index.map(dias_es)

# Crea gráfico de barras con días en español
fig = px.bar(
    x=conteo_dias.index,
    y=conteo_dias.values,
    labels={'x': 'Día de la semana', 'y': 'Número de accidentes'},
    title='Accidentes por Día de la Semana'
)

# Gira ligeramente las etiquetas del eje X para mayor legibilidad
fig.update_layout(xaxis_tickangle=-30)

# Muestra el gráfico
fig.show()


- Se observan más accidentes los **viernes** y **sábados**, lo cual es coherente con un aumento en la movilidad por actividades sociales y laborales.

In [4]:
# Gráfico de accidentes por hora del día

# Crea una nueva columna 'hora_num' con la hora como número entero (0 a 23)
df['hora_num'] = pd.to_datetime(df['hora_evento'].astype(str), format='%H:%M:%S', errors='coerce').dt.hour

# Cuenta la cantidad de accidentes por cada hora y ordena de 0 a 23
conteo_horas = df['hora_num'].value_counts().sort_index()

# Crea gráfico de barras con las horas como eje X
fig = px.bar(
    x=conteo_horas.index,
    y=conteo_horas.values,
    labels={'x': 'Hora del día', 'y': 'Número de accidentes'},
    title='Distribución de accidentes por hora'
)

# Asegura que se muestre cada hora como una marca en el eje X
fig.update_layout(xaxis=dict(dtick=1))

# Muestra el gráfico
fig.show()



- Claros picos de accidentes a las **8:00 AM** y entre las **16:00 y 19:00 hrs**, correspondientes a las horas pico del tránsito.


In [5]:
# Gráfico de accidentes por alcaldía

# Cuenta la cantidad de accidentes por alcaldía
conteo_alcaldias = df['alcaldia'].value_counts()

# Crea gráfico de barras con nombres de alcaldías en el eje X
fig = px.bar(
    x=conteo_alcaldias.index,
    y=conteo_alcaldias.values,
    title="Accidentes por Alcaldía",
    labels={'x': 'Alcaldía', 'y': 'Número de accidentes'}
)

# Ajusta títulos de ejes y gira etiquetas del eje X para mejor lectura
fig.update_layout(
    xaxis_title='Alcaldía',
    yaxis_title='Número de accidentes',
    xaxis_tickangle=-45
)

# Muestra el gráfico
fig.show()


- Las alcaldías con más accidentes son **Cuauhtémoc**, **Iztapalapa** y **Gustavo A. Madero**. Estas zonas suelen tener alta densidad vehicular y actividad comercial.

In [6]:
# Gráfico: Top 20 colonias con más accidentes

# Obtiene las 20 colonias con mayor cantidad de accidentes
top_colonias = df['colonia'].value_counts().head(20)

# Crea gráfico de barras con nombres de colonias en el eje X
fig = px.bar(
    x=top_colonias.index,
    y=top_colonias.values,
    title="Top 20 Colonias con más Accidentes",
    labels={'x': 'Colonia', 'y': 'Número de accidentes'}
)

# Ajusta títulos y gira etiquetas del eje X para mejor lectura
fig.update_layout(
    xaxis_title='Colonia',
    yaxis_title='Número de accidentes',
    xaxis_tickangle=-45
)

# Muestra el gráfico
fig.show()


- Las colonias con más accidentes son **Agrícola Oriental**, **Centro** y **Juárez**. Representan puntos críticos a considerar en futuras visualizaciones con mapa.


In [7]:
# Gráfico de frecuencia por tipo de evento

# Cuenta la cantidad de accidentes por cada tipo de evento (ej. Choque, Atropellado)
conteo_eventos = df['tipo_evento'].value_counts()

# Crea gráfico de barras con los tipos de evento en el eje X
fig = px.bar(
    x=conteo_eventos.index,
    y=conteo_eventos.values,
    title="Frecuencia por Tipo de Evento",
    labels={'x': 'Tipo de evento', 'y': 'Número de accidentes'}
)

# Ajustes de presentación: ejes, ángulo de etiquetas y sin leyenda
fig.update_layout(
    xaxis_title='Tipo de evento',
    yaxis_title='Número de accidentes',
    xaxis_tickangle=-30,
    showlegend=False  # Oculta la leyenda (innecesaria aquí)
)

# Muestra el gráfico
fig.show()

- El evento más frecuente es el **choque**, seguido por **derrapado** y **atropellado**. Esto sugiere que la mayoría de los incidentes involucran vehículos en movimiento, no peatones.

In [8]:
# Gráfico de distribución de prioridad de atención

# Crea un gráfico circular (pie chart) mostrando la proporción de cada nivel de prioridad
fig = px.pie(
    df,
    names='prioridad',  # Agrupa los datos por la columna 'prioridad'
    title="Distribución de Prioridad de Atención",
    color_discrete_sequence=px.colors.qualitative.Set2  # Colores suaves y diferenciados
)

# Ajustes visuales del gráfico
fig.update_traces(
    textinfo='label+percent',         # Muestra etiqueta + porcentaje en cada segmento
    textposition='outside',           # Ubica los textos fuera del círculo para mayor legibilidad
    insidetextorientation='radial',   # Orientación interna del texto si fuera necesario
    marker=dict(
        line=dict(color='white', width=2)  # Bordes blancos para separar bien los segmentos
    )
)

# Muestra el gráfico
fig.show()


- Más del **80%** de los accidentes se clasifican como de **prioridad baja**, con muy pocos casos en nivel alto. Esto podría reflejar que la mayoría no son eventos graves, o un subregistro en los más severos.
